# インポート

In [1]:
import preprocessing
from feature_engineering import FeatureCreator
#from feature_engineering import FeatureCreator, PredictionFeatureCreator
from train import Trainer
import prediction
from evaluation import Evaluator
%load_ext autoreload

In [11]:
%autoreload

# データ加工

In [12]:
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()

In [13]:
# 重複チェック
results_preprocessed.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [14]:
# 欠損チェック
results_preprocessed.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
rank           0
wakuban        0
umaban         0
sex            0
age            0
weight         0
weight_diff    0
tansho_odds    0
popularity     0
impost         0
dtype: int64

In [15]:
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()

In [16]:
# 欠損値チェック
horse_results_preprocessed.isnull().sum()

horse_id            0
date                0
rank                0
price               0
rank_diff        2248
weather           394
race_type           0
course_len          0
ground_state      179
race_class      62866
time             2178
win                 0
rentai              0
show                0
place               0
n_horses            0
dtype: int64

In [17]:
# 重複チェック
horse_results_preprocessed.duplicated(subset=["horse_id", "date"]).sum()

np.int64(0)

In [18]:
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [19]:
race_info_preprocessed

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place,month,sin_date,cos_date
0,202301010101,2023-07-22,1,0.0,1200,0,0,1,1,7,0.656633,0.060799
1,202301010102,2023-07-22,0,0.0,1000,0,0,1,1,7,0.656633,0.060799
2,202301010103,2023-07-22,0,0.0,1700,0,0,1,1,7,0.656633,0.060799
3,202301010104,2023-07-22,1,0.0,1500,0,0,1,1,7,0.656633,0.060799
4,202301010105,2023-07-22,0,0.0,1700,0,0,0,1,7,0.656633,0.060799
...,...,...,...,...,...,...,...,...,...,...,...,...
3451,202310030808,2023-09-03,0,0.0,1000,0,0,2,10,9,0.113279,0.537695
3452,202310030809,2023-09-03,1,0.0,1200,0,0,3,10,9,0.113279,0.537695
3453,202310030810,2023-09-03,0,0.0,1700,0,0,4,10,9,0.113279,0.537695
3454,202310030811,2023-09-03,1,0.0,1200,0,0,7,10,9,0.113279,0.537695


In [20]:
# 払い戻しテーブルの前処理
return_tables_preprocessed = preprocessing.process_return_tables()

In [21]:
return_tables_preprocessed

,bet_type,win_umaban,return
race_id,,,
202301010101,単勝,[5],120
202301010101,複勝,[5],100
202301010101,複勝,[8],110
202301010101,複勝,[6],310
202301010101,馬連,"[5, 8]",170
...,...,...,...
202310030812,ワイド,"[6, 8]",410
202310030812,ワイド,"[6, 11]",470
202310030812,馬単,"[8, 11]",1580


# 特徴量の作成

In [2]:
fc = FeatureCreator()
features = fc.create_features()

In [3]:
fc.population

,race_id,horse_id,date
0,202301010101,2021100648,2023-07-22
1,202301010101,2021100159,2023-07-22
2,202301010101,2021100265,2023-07-22
3,202301010101,2021105553,2023-07-22
4,202301010101,2021101429,2023-07-22
...,...,...,...
47268,202310030812,2019102542,2023-09-03
47269,202310030812,2020103754,2023-09-03
47270,202310030812,2020100400,2023-09-03
47271,202310030812,2020103663,2023-09-03


In [4]:
fc.population.race_id.nunique()

3456

In [5]:
fc.horse_results

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,time,win,rentai,show,place,n_horses
0,2018102264,2024-01-21,10,0.0,1.0,1.0,0,1200,1.0,4.0,71.2,0,0,0,京都,16
1,2018102264,2023-12-09,5,184.0,0.5,0.0,0,1200,0.0,4.0,71.2,0,0,0,中山,16
2,2018102264,2023-07-23,4,275.0,0.2,0.0,0,1150,0.0,5.0,67.4,0,0,0,福島,11
3,2018102264,2023-06-11,7,0.0,0.3,1.0,0,1200,1.0,4.0,71.1,0,0,0,阪神,16
4,2018102264,2023-04-02,9,0.0,1.5,0.0,0,1200,0.0,4.0,73.0,0,0,0,阪神,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161380,2020102750,2023-12-27,1,40.0,0.0,0.0,0,1400,0.0,NaN,93.1,1,1,1,笠松,7
161381,2020102750,2023-12-06,3,8.0,0.4,0.0,0,1400,0.0,NaN,90.8,0,0,1,笠松,8
161382,2020102750,2023-11-07,5,2.0,2.3,1.0,0,800,1.0,NaN,51.1,0,0,0,笠松,7
161383,2020102750,2023-09-03,14,0.0,3.4,0.0,0,1700,0.0,1.0,109.4,0,0,0,小倉,16


In [7]:
df = (
    fc.population
    .merge(fc.horse_results, on="horse_id", suffixes=("", "_horse"))
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
    .groupby(["race_id", "horse_id"])
    .head(5)
)
df

,race_id,horse_id,date,date_horse,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,time,win,rentai,show,place,n_horses
551240,202309050903,2021105731,2023-12-28,2023-12-23,12,0.0,1.6,0.0,1,1800,0.0,1.0,109.9,0,0,0,阪神,14
551237,202309050903,2021102811,2023-12-28,2023-12-23,17,0.0,3.3,0.0,1,1400,0.0,0.0,84.3,0,0,0,阪神,17
551304,202309050903,2021104215,2023-12-28,2023-12-23,14,0.0,2.0,0.0,1,1800,0.0,1.0,110.3,0,0,0,阪神,14
551351,202309050903,2021105018,2023-12-28,2023-12-23,14,0.0,3.3,0.0,0,1200,0.0,1.0,76.9,0,0,0,阪神,15
248873,202306050902,2021100360,2023-12-28,2023-12-17,15,0.0,NaN,0.0,0,1800,0.0,1.0,NaN,0,0,0,中山,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90068,202305020609,2015101660,2023-05-07,2020-01-25,1,760.0,0.0,1.0,1,1600,0.0,2.0,94.7,1,1,1,中山,16
116483,202305030610,2015101660,2023-06-18,2020-01-25,1,760.0,0.0,1.0,1,1600,0.0,2.0,94.7,1,1,1,中山,16
90069,202305020609,2015101660,2023-05-07,2018-08-18,2,430.4,0.2,0.0,1,1800,0.0,2.0,109.3,0,1,1,札幌,11
116484,202305030610,2015101660,2023-06-18,2018-08-18,2,430.4,0.2,0.0,1,1800,0.0,2.0,109.3,0,1,1,札幌,11


In [8]:
df.groupby(["race_id", "horse_id"])[["rank", "prize"]].mean()

rank   prize
race_id      horse_id                
202303020811 2014102562   7.6  177.52
             2016110131   6.2  440.86
             2018101878   7.2  558.10
             2018101984   7.4  266.12
             2018102264   7.2   92.80
...                       ...     ...
202310030812 2020103324   7.0   28.00
             2020103663   6.2   97.00
             2020103754   6.0  145.00
             2020105644   7.6   27.60
             2020106455   7.2   57.60

[23091 rows x 2 columns]

In [9]:
fc.horse_results

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,time,win,rentai,show,place,n_horses
0,2018102264,2024-01-21,10,0.0,1.0,1.0,0,1200,1.0,4.0,71.2,0,0,0,京都,16
1,2018102264,2023-12-09,5,184.0,0.5,0.0,0,1200,0.0,4.0,71.2,0,0,0,中山,16
2,2018102264,2023-07-23,4,275.0,0.2,0.0,0,1150,0.0,5.0,67.4,0,0,0,福島,11
3,2018102264,2023-06-11,7,0.0,0.3,1.0,0,1200,1.0,4.0,71.1,0,0,0,阪神,16
4,2018102264,2023-04-02,9,0.0,1.5,0.0,0,1200,0.0,4.0,73.0,0,0,0,阪神,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161380,2020102750,2023-12-27,1,40.0,0.0,0.0,0,1400,0.0,NaN,93.1,1,1,1,笠松,7
161381,2020102750,2023-12-06,3,8.0,0.4,0.0,0,1400,0.0,NaN,90.8,0,0,1,笠松,8
161382,2020102750,2023-11-07,5,2.0,2.3,1.0,0,800,1.0,NaN,51.1,0,0,0,笠松,7
161383,2020102750,2023-09-03,14,0.0,3.4,0.0,0,1700,0.0,1.0,109.4,0,0,0,小倉,16


In [19]:
agg_df = (
    df.groupby(["race_id", "horse_id"])[
        ["rank", "prize", "rank_diff", "race_class"]
    ].agg(["mean", "median", "max", "min"])
)
agg_df

rank                  prize                      \
                        mean median max min    mean median     max  min   
race_id      horse_id                                                     
202303020811 2014102562  7.6    8.0  13   2  177.52    0.0   887.6  0.0   
             2016110131  6.2    6.0  12   1  440.86    0.0  1874.3  0.0   
             2018101878  7.2    7.0  14   1  558.10    0.0  2237.1  0.0   
             2018101984  7.4    5.0  13   3  266.12  220.0   555.8  0.0   
             2018102264  7.2    7.0  11   4   92.80    0.0   280.0  0.0   
...                      ...    ...  ..  ..     ...    ...     ...  ...   
202310030812 2020103324  7.0    7.0  10   3   28.00    0.0   140.0  0.0   
             2020103663  6.2    5.0  10   4   97.00   78.0   300.0  0.0   
             2020103754  6.0    5.0  13   1  145.00   55.0   550.0  0.0   
             2020105644  7.6    8.0  13   4   27.60    0.0    83.0  0.0   
             2020106455  7.2    7.0  12   2   57.60    0.0   210.0  0.0   

                        rank_diff                  race_class                   
                             mean median  max  min       mean median  max  min  
race_id      horse_id                                                           
202303020811 2014102562      0.76    0.5  1.5  0.0        5.0    5.0  5.0  5.0  
             2016110131      0.78    0.7  1.3  0.0        4.4    4.0  5.0  4.0  
             2018101878      1.04    1.1  2.3  0.0        5.6    5.0  7.0  5.0  
             2018101984      0.92    0.6  2.2  0.3        5.0    5.0  5.0  5.0  
             2018102264      0.74    0.6  1.5  0.3        4.0    4.0  4.0  4.0  
...                           ...    ...  ...  ...        ...    ...  ...  ...  
202310030812 2020103324      1.08    1.1  2.5  0.0        1.0    1.0  1.0  1.0  
             2020103663      1.14    1.3  2.4  0.3        2.8    2.0  6.0  2.0  
             2020103754      1.18    1.3  2.3  0.0        1.6    2.0  2.0  1.0  
             2020105644      1.22    0.8  2.3  0.6        1.0    1.0  1.0  1.0  
             2020106455      1.04    1.1  2.3  0.2        0.8    1.0  1.0  0.0  

[23091 rows x 16 columns]

In [20]:
agg_df.columns

MultiIndex([(      'rank',   'mean'),
            (      'rank', 'median'),
            (      'rank',    'max'),
            (      'rank',    'min'),
            (     'prize',   'mean'),
            (     'prize', 'median'),
            (     'prize',    'max'),
            (     'prize',    'min'),
            ( 'rank_diff',   'mean'),
            ( 'rank_diff', 'median'),
            ( 'rank_diff',    'max'),
            ( 'rank_diff',    'min'),
            ('race_class',   'mean'),
            ('race_class', 'median'),
            ('race_class',    'max'),
            ('race_class',    'min')],
           )

In [14]:
"_".join(agg_df.columns[0])

'rank_mean'

In [21]:
agg_df.columns = ["_".join(col) for col in agg_df.columns]
agg_df

rank_mean  rank_median  rank_max  rank_min  \
race_id      horse_id                                                 
202303020811 2014102562        7.6          8.0        13         2   
             2016110131        6.2          6.0        12         1   
             2018101878        7.2          7.0        14         1   
             2018101984        7.4          5.0        13         3   
             2018102264        7.2          7.0        11         4   
...                            ...          ...       ...       ...   
202310030812 2020103324        7.0          7.0        10         3   
             2020103663        6.2          5.0        10         4   
             2020103754        6.0          5.0        13         1   
             2020105644        7.6          8.0        13         4   
             2020106455        7.2          7.0        12         2   

                         prize_mean  prize_median  prize_max  prize_min  \
race_id      horse_id                                                     
202303020811 2014102562      177.52           0.0      887.6        0.0   
             2016110131      440.86           0.0     1874.3        0.0   
             2018101878      558.10           0.0     2237.1        0.0   
             2018101984      266.12         220.0      555.8        0.0   
             2018102264       92.80           0.0      280.0        0.0   
...                             ...           ...        ...        ...   
202310030812 2020103324       28.00           0.0      140.0        0.0   
             2020103663       97.00          78.0      300.0        0.0   
             2020103754      145.00          55.0      550.0        0.0   
             2020105644       27.60           0.0       83.0        0.0   
             2020106455       57.60           0.0      210.0        0.0   

                         rank_diff_mean  rank_diff_median  rank_diff_max  \
race_id      horse_id                                                      
202303020811 2014102562            0.76               0.5            1.5   
             2016110131            0.78               0.7            1.3   
             2018101878            1.04               1.1            2.3   
             2018101984            0.92               0.6            2.2   
             2018102264            0.74               0.6            1.5   
...                                 ...               ...            ...   
202310030812 2020103324            1.08               1.1            2.5   
             2020103663            1.14               1.3            2.4   
             2020103754            1.18               1.3            2.3   
             2020105644            1.22               0.8            2.3   
             2020106455            1.04               1.1            2.3   

                         rank_diff_min  race_class_mean  race_class_median  \
race_id      horse_id                                                        
202303020811 2014102562            0.0              5.0                5.0   
             2016110131            0.0              4.4                4.0   
             2018101878            0.0              5.6                5.0   
             2018101984            0.3              5.0                5.0   
             2018102264            0.3              4.0                4.0   
...                                ...              ...                ...   
202310030812 2020103324            0.0              1.0                1.0   
             2020103663            0.3              2.8                2.0   
             2020103754            0.0              1.6                2.0   
             2020105644            0.6              1.0                1.0   
             2020106455            0.2              0.8                1.0   

                         race_class_max  race_class_min  
race_id      horse_id                                    
20230

In [23]:
fc.population.merge(
    agg_df, on=["race_id", "horse_id"], how="left"
)

,race_id,horse_id,date,rank_mean,rank_median,rank_max,rank_min,prize_mean,prize_median,prize_max,prize_min,rank_diff_mean,rank_diff_median,rank_diff_max,rank_diff_min,race_class_mean,race_class_median,race_class_max,race_class_min
0,202301010101,2021100648,2023-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202301010101,2021100159,2023-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202301010101,2021100265,2023-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202301010101,2021105553,2023-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202301010101,2021101429,2023-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,202310030812,2019102542,2023-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47269,202310030812,2020103754,2023-09-03,6.0,5.0,13.0,1.0,145.0,55.0,550.0,0.0,1.18,1.3,2.3,0.0,1.6,2.0,2.0,1.0
47270,202310030812,2020100400,2023-09-03,8.8,9.0,11.0,6.0,0.0,0.0,0.0,0.0,1.90,1.8,3.0,1.0,1.0,1.0,1.0,1.0
47271,202310030812,2020103663,2023-09-03,6.2,5.0,10.0,4.0,97.0,78.0,300.0,0.0,1.14,1.3,2.4,0.3,2.8,2.0,6.0,2.0


## 標準化
$$
z = \frac{x- \mu}{\sigma}
$$

- z は標準化後の値
- x はもとのデータ値
- 𝜇 はデータの平均値
- 𝜎 はデータの標準偏差

ちなみに、これを使って以下の式に変化したものが「偏差値」

$$
T = 50 + 10 \times \frac{x- \mu}{\sigma}
$$

In [24]:
agg_df.groupby("race_id").mean()

,rank_mean,rank_median,rank_max,rank_min,prize_mean,prize_median,prize_max,prize_min,rank_diff_mean,rank_diff_median,rank_diff_max,rank_diff_min,race_class_mean,race_class_median,race_class_max,race_class_min
race_id,,,,,,,,,,,,,,,,
202303020811,7.233333,6.833333,12.666667,2.000000,318.376667,36.666667,1284.850000,0.000000,0.883333,0.800000,1.800000,0.100000,4.766667,4.500000,5.333333,4.500000
202303020812,4.300000,3.000000,13.000000,1.000000,146.750000,121.250000,340.000000,0.000000,0.600000,0.300000,2.200000,0.000000,1.500000,1.500000,1.500000,1.500000
202303030101,8.111111,8.111111,8.777778,7.444444,44.444444,44.444444,44.444444,44.444444,1.883333,1.861111,2.011111,1.777778,0.129630,0.166667,0.222222,0.000000
202303030102,7.850000,7.800000,9.200000,6.600000,22.450000,22.450000,37.700000,7.200000,1.981667,1.835000,2.620000,1.500000,0.400000,0.450000,0.700000,0.000000
202303030103,6.300000,4.000000,13.875000,1.875000,116.675000,101.500000,333.750000,0.000000,1.135000,0.712500,2.837500,0.125000,1.775000,1.875000,2.000000,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310030808,6.871795,6.923077,11.000000,2.461538,77.620513,10.923077,282.307692,2.307692,0.958205,0.950000,1.623077,0.338462,1.677778,1.750000,2.000000,1.333333
202310030809,6.781818,6.272727,11.090909,3.181818,147.643636,79.200000,398.927273,18.181818,0.690000,0.631818,1.290909,0.236364,3.090909,3.090909,3.636364,2.727273
202310030810,7.875000,8.375000,12.750000,3.125000,166.712500,37.250000,561.162500,0.000000,1.162500,1.087500,2.087500,0.312500,3.875000,3.875000,4.000000,3.625000


In [ ]:
# 標準偏差
agg_df.groupby("race_id").std()

,rank_mean,rank_median,rank_max,rank_min,prize_mean,prize_median,prize_max,prize_min,rank_diff_mean,rank_diff_median,rank_diff_max,rank_diff_min,race_class_mean,race_class_median,race_class_max,race_class_min
race_id,,,,,,,,,,,,,,,,
202303020811,0.557375,1.169045,1.032796,1.264911,172.693951,89.814624,812.493404,0.000000,0.143898,0.322490,0.419524,0.154919,0.557375,0.547723,1.032796,0.547723
202303020812,1.272792,1.414214,4.242641,0.000000,131.875415,139.653589,296.984848,0.000000,0.311127,0.141421,1.838478,0.000000,0.707107,0.707107,0.707107,0.707107
202303030101,4.044887,4.044887,4.603742,3.940530,99.008978,99.008978,99.008978,99.008978,1.176860,1.182629,1.217009,1.208075,0.260579,0.353553,0.440959,0.000000
202303030102,2.550054,2.496664,2.859681,2.836273,35.391815,35.391815,53.433344,22.768399,1.109734,0.964955,1.782508,1.131371,0.293447,0.368932,0.483046,0.000000
202303030103,2.002855,3.251373,2.900123,2.100170,75.438447,72.748098,264.355686,0.000000,0.572289,0.903070,0.982617,0.315096,0.328416,0.353553,0.000000,0.755929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310030808,2.274571,3.353146,4.123106,1.613246,55.953283,24.746924,214.015336,8.320503,0.379724,0.565317,0.643109,0.386304,0.466955,0.452267,1.044466,0.778499
202310030809,2.096577,1.555050,3.207945,1.662419,126.378337,124.139116,259.196513,60.302269,0.230695,0.253252,0.476350,0.201359,0.826383,1.044466,1.689540,0.646670
202310030810,2.319945,2.924649,3.453776,1.959410,123.722246,71.465776,407.726582,0.000000,0.478502,0.375832,0.780911,0.511126,0.281577,0.353553,0.000000,0.744024


In [26]:
relative_df = (agg_df - agg_df.groupby("race_id").mean()) / agg_df.groupby("race_id").std()
relative_df

rank_mean  rank_median  rank_max  rank_min  \
race_id      horse_id                                                 
202303020811 2014102562   0.657846     0.997965  0.322749  0.000000   
             2016110131  -1.853929    -0.712832 -0.645497 -0.790569   
             2018101878  -0.059804     0.142566  1.290994 -0.790569   
             2018101984   0.299021    -1.568231  0.322749  0.790569   
             2018102264  -0.059804     0.142566 -1.613743  1.581139   
...                            ...          ...       ...       ...   
202310030812 2020103324   0.011216     0.041107 -0.492877 -0.123608   
             2020103663  -0.482307    -0.863241 -0.492877  0.556234   
             2020103754  -0.605687    -0.863241  0.591453 -1.483291   
             2020105644   0.381359     0.493280  0.591453  0.556234   
             2020106455   0.134597     0.041107  0.230009 -0.803449   

                         prize_mean  prize_median  prize_max  prize_min  \
race_id      horse_id                                                     
202303020811 2014102562   -0.815643     -0.408248  -0.488927        NaN   
             2016110131    0.709251     -0.408248   0.725483        NaN   
             2018101878    1.388140     -0.408248   1.172010        NaN   
             2018101984   -0.302597      2.041241  -0.897300        NaN   
             2018102264   -1.306222     -0.408248  -1.236749        NaN   
...                             ...           ...        ...        ...   
202310030812 2020103324   -0.372169     -0.572692  -0.101815        NaN   
             2020103663    1.174459      1.702168   0.919232        NaN   
             2020103754    2.250373      1.031376   2.514616        NaN   
             2020105644   -0.381135     -0.572692  -0.465562        NaN   
             2020106455    0.291312     -0.572692   0.344893        NaN   

                         rank_diff_mean  rank_diff_median  rank_diff_max  \
race_id      horse_id                                                      
202303020811 2014102562       -0.857088         -0.930261      -0.715097   
             2016110131       -0.718101         -0.310087      -1.191828   
             2018101878        1.088733          0.930261       1.191828   
             2018101984        0.254810         -0.620174       0.953463   
             2018102264       -0.996075         -0.620174      -0.715097   
...                                 ...               ...            ...   
202310030812 2020103324       -0.267686         -0.217186       0.285893   
             2020103663       -0.149904          0.217186       0.173578   
             2020103754       -0.071383          0.217186       0.061263   
             2020105644        0.007138         -0.868744       0.061263   
             2020106455       -0.346207         -0.217186       0.061263   

                         rank_diff_min  race_class_mean  race_class_median  \
race_id      horse_id                                                        
202303020811 2014102562      -0.645497         0.418629           0.912871   
             2016110131      -0.645497        -0.657846          -0.912871   
             2018101878      -0.645497         1.495104           0.912871   
             2018101984       1.290994         0.418629           0.912871   
             2018102264       1.290994        -1.375496          -0.912871   
...                                ...              ...                ...   
202310030812 2020103324      -1.000511        -0.731018          -0.870388   
             2020103663      -0.214395         2.052474           1.044466   
             2020103754      -1.000511         0.196813           1.044466   
             2020105644       0.571720        -0.731018          -0.870388   
             2020106455      -0.476434        -1.040295          -0.870388   

                         race_class_max  race_class_min  
race_id      horse_id                                    
20230

In [27]:
relative_df.head(10)

rank_mean  rank_median  rank_max  rank_min  \
race_id      horse_id                                                 
202303020811 2014102562   0.657846     0.997965  0.322749  0.000000   
             2016110131  -1.853929    -0.712832 -0.645497 -0.790569   
             2018101878  -0.059804     0.142566  1.290994 -0.790569   
             2018101984   0.299021    -1.568231  0.322749  0.790569   
             2018102264  -0.059804     0.142566 -1.613743  1.581139   
             2018110126   1.016671     0.997965  0.322749 -0.790569   
202303020812 2019106339   0.707107     0.707107  0.707107       NaN   
             2020103366  -0.707107    -0.707107 -0.707107       NaN   
202303030101 2021101940  -0.027470    -0.027470 -0.168945  0.140985   
             2021102352  -1.510824    -1.510824 -1.472232 -1.381653   

                         prize_mean  prize_median  prize_max  prize_min  \
race_id      horse_id                                                     
202303020811 2014102562   -0.815643     -0.408248  -0.488927        NaN   
             2016110131    0.709251     -0.408248   0.725483        NaN   
             2018101878    1.388140     -0.408248   1.172010        NaN   
             2018101984   -0.302597      2.041241  -0.897300        NaN   
             2018102264   -1.306222     -0.408248  -1.236749        NaN   
             2018110126    0.327072     -0.408248   0.725483        NaN   
202303020812 2019106339   -0.707107     -0.707107  -0.707107        NaN   
             2020103366    0.707107      0.707107   0.707107        NaN   
202303030101 2021101940   -0.448893     -0.448893  -0.448893  -0.448893   
             2021102352    2.480134      2.480134   2.480134   2.480134   

                         rank_diff_mean  rank_diff_median  rank_diff_max  \
race_id      horse_id                                                      
202303020811 2014102562       -0.857088         -0.930261      -0.715097   
             2016110131       -0.718101         -0.310087      -1.191828   
             2018101878        1.088733          0.930261       1.191828   
             2018101984        0.254810         -0.620174       0.953463   
             2018102264       -0.996075         -0.620174      -0.715097   
             2018110126        1.227721          1.550434       0.476731   
202303020812 2019106339        0.707107         -0.707107       0.707107   
             2020103366       -0.707107          0.707107      -0.707107   
202303030101 2021101940        0.099134          0.117441      -0.009130   
             2021102352       -1.260416         -1.235477      -1.323828   

                         rank_diff_min  race_class_mean  race_class_median  \
race_id      horse_id                                                        
202303020811 2014102562      -0.645497         0.418629           0.912871   
             2016110131      -0.645497        -0.657846          -0.912871   
             2018101878      -0.645497         1.495104           0.912871   
             2018101984       1.290994         0.418629           0.912871   
             2018102264       1.290994        -1.375496          -0.912871   
             2018110126      -0.645497        -0.299021          -0.912871   
202303020812 2019106339            NaN         0.707107           0.707107   
             2020103366            NaN        -0.707107          -0.707107   
202303030101 2021101940       0.183947        -0.497468          -0.471405   
             2021102352      -1.140474        -0.497468          -0.471405   

                         race_class_max  race_class_min  
race_id      horse_id                                    
202303020811 2014102562       -0.322749        0.912871  
             2016110131       -0.322749       -0.912871  
             2018101878        1.613743        0.912871  
             2018101984       -0.322749        0.912871  
             2018102264       -1.290994       -0.912871  
             20

In [28]:
agg_df.head(10)

rank_mean  rank_median  rank_max  rank_min  \
race_id      horse_id                                                 
202303020811 2014102562        7.6          8.0        13         2   
             2016110131        6.2          6.0        12         1   
             2018101878        7.2          7.0        14         1   
             2018101984        7.4          5.0        13         3   
             2018102264        7.2          7.0        11         4   
             2018110126        7.8          8.0        13         1   
202303020812 2019106339        5.2          4.0        16         1   
             2020103366        3.4          2.0        10         1   
202303030101 2021101940        8.0          8.0         8         8   
             2021102352        2.0          2.0         2         2   

                         prize_mean  prize_median  prize_max  prize_min  \
race_id      horse_id                                                     
202303020811 2014102562      177.52           0.0      887.6        0.0   
             2016110131      440.86           0.0     1874.3        0.0   
             2018101878      558.10           0.0     2237.1        0.0   
             2018101984      266.12         220.0      555.8        0.0   
             2018102264       92.80           0.0      280.0        0.0   
             2018110126      374.86           0.0     1874.3        0.0   
202303020812 2019106339       53.50          22.5      130.0        0.0   
             2020103366      240.00         220.0      550.0        0.0   
202303030101 2021101940        0.00           0.0        0.0        0.0   
             2021102352      290.00         290.0      290.0      290.0   

                         rank_diff_mean  rank_diff_median  rank_diff_max  \
race_id      horse_id                                                      
202303020811 2014102562            0.76               0.5            1.5   
             2016110131            0.78               0.7            1.3   
             2018101878            1.04               1.1            2.3   
             2018101984            0.92               0.6            2.2   
             2018102264            0.74               0.6            1.5   
             2018110126            1.06               1.3            2.0   
202303020812 2019106339            0.82               0.2            3.5   
             2020103366            0.38               0.4            0.9   
202303030101 2021101940            2.00               2.0            2.0   
             2021102352            0.40               0.4            0.4   

                         rank_diff_min  race_class_mean  race_class_median  \
race_id      horse_id                                                        
202303020811 2014102562            0.0              5.0                5.0   
             2016110131            0.0              4.4                4.0   
             2018101878            0.0              5.6                5.0   
             2018101984            0.3              5.0                5.0   
             2018102264            0.3              4.0                4.0   
             2018110126            0.0              4.6                4.0   
202303020812 2019106339            0.0              2.0                2.0   
             2020103366            0.0              1.0                1.0   
202303030101 2021101940            2.0              0.0                0.0   
             2021102352            0.4              0.0                0.0   

                         race_class_max  race_class_min  
race_id      horse_id                                    
202303020811 2014102562             5.0             5.0  
             2016110131             5.0             4.0  
             2018101878             7.0             5.0  
             2018101984             5.0             5.0  
             2018102264             4.0             4.0  
             20

In [30]:
(177.52 + 440.86 + 558.10 + 266.12 + 92.80 + 374.86) / 6

318.3766666666666

In [31]:
(374.86 - 318.3766666666666)/ 172.693951

0.32707186908552116

In [23]:
# 重複チェック
features.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

# 学習

In [24]:
trainer = Trainer()
evaluation_df = trainer.run(test_start_date="2023-10-01")

[100]	training's binary_logloss: 0.15511	valid_1's binary_logloss: 0.208311


# 精度評価

In [25]:
evaluator = Evaluator()
evaluator.summarize_box_top_n(n=1, save_filename="box_summary_top1.csv")

,hitrate_model,hitrate_pop,returnrate_model,returnrate_pop
bet_type,,,,
単勝,0.321759,0.331019,0.825116,0.798727
複勝,0.616898,0.641204,0.826736,0.838194


In [26]:
evaluator.summarize_box_top_n(n=2, save_filename="box_summary_top2.csv")

,hitrate_model,hitrate_pop,returnrate_model,returnrate_pop
bet_type,,,,
ワイド,0.319444,0.324074,0.854167,0.809606
単勝,0.545139,0.543981,0.881192,0.836285
複勝,0.837963,0.842593,0.856655,0.839410
馬単,0.159722,0.170139,0.861574,0.811748
馬連,0.159722,0.170139,0.865394,0.829977


In [27]:
evaluator.summarize_box_top_n(n=3, save_filename="box_summary_top3.csv")

,hitrate_model,hitrate_pop,returnrate_model,returnrate_pop
bet_type,,,,
ワイド,0.535880,0.559028,0.806250,0.802353
三連単,0.092593,0.093750,0.660031,0.607870
三連複,0.092593,0.093750,0.801620,0.731713
単勝,0.652778,0.663194,0.813349,0.794676
複勝,0.927083,0.924769,0.837616,0.824807
馬単,0.315972,0.322917,0.741281,0.713117
馬連,0.315972,0.322917,0.770062,0.733719


# 予測

当日出走馬が確定した時点で実行できる

In [28]:
# 当日出走馬の過去成績テーブルの前処理
horse_results_preprocessed = preprocessing.process_horse_results(
    input_filename="horse_results_prediction.csv",
    output_filename="horse_results_prediction.csv"
)

In [29]:
# 予測
prediction.predict(features)

,race_id,umaban,tansho_odds,popularity,pred
19258,202305050812,2,1.3,1,0.943584
31977,202308010206,3,1.2,1,0.904607
6284,202303030103,3,3.5,1,0.891395
13210,202305020109,2,1.2,1,0.866842
42252,202309050202,4,1.5,1,0.861527
...,...,...,...,...,...
14289,202305020712,7,429.9,15,0.000223
39079,202309020606,3,354.9,13,0.000222
36466,202309010201,3,395.0,14,0.000221
40794,202309030811,16,480.4,17,0.000218
